In [ ]:
%pip install pandas
%pip install pysqlite3
%pip install vanna
%pip install openai
%pip install chromadb
%pip install chardet
%pip install --upgrade requests
%pip install --user onnxruntime-gpu

In [ ]:
%pip install ollama

## 1. Import Sample Data

In [2]:
import pandas as pd
df = pd.read_csv('data/tips.csv')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## Create Sample Database

In [3]:
import sqlite3
conn = sqlite3.connect('data/tips.db')
df.to_sql('tips', conn, if_exists='replace', index=False)

244

## 2. Obtain Database Query As Training Dataset

In [4]:
table_name = 'tips'
cursor = conn.cursor()
query = f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table_name}'"
cursor.execute(query)
create_table_sql = cursor.fetchone()[0]
print(create_table_sql)
cursor.close()
conn.close()

CREATE TABLE "tips" (
"total_bill" REAL,
  "tip" REAL,
  "sex" TEXT,
  "smoker" TEXT,
  "day" TEXT,
  "time" TEXT,
  "size" INTEGER
)


## 3. Connect To Your Original Model In Ollama
In `MyVanna` config, `'model'` should be the exact name you find using command `ollama list`.

In [5]:
import vanna.ollama.ollama as vanna_ollama
import requests

def _final_submit_prompt(self, prompt, **kwargs):
    if isinstance(prompt, list):
        prompt_str = ""
        for msg in prompt:
            role = msg.get("role", "")
            content = msg.get("content", "")
            prompt_str += f"{role}: {content}\n"
    else:
        prompt_str = str(prompt)

    url = "http://localhost:11436/api/generate"

    try:
        res = requests.post(
            url,
            json={
                "model": "llama3.2",
                "prompt": prompt_str,
                "stream": False
            },
            timeout=60
        )
        res.raise_for_status()
        response_dict = res.json()
        return response_dict.get("response", "")
    except Exception as e:
        print("Connect to Ollama Fail", e)
        raise e

vanna_ollama.Ollama.submit_prompt = _final_submit_prompt


In [6]:
import vanna.ollama.ollama as vanna_ollama
vanna_ollama.Ollama._Ollama__pull_model_if_ne = staticmethod(lambda *args, **kwargs: None)

from vanna.ollama import Ollama
from vanna.chromadb.chromadb_vector import ChromaDB_VectorStore
from vanna.flask import VannaFlaskApp

class MyVanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

vn = MyVanna(config={
    'temperature': 0.5,
    'model': 'llama3.2',
    'host': 'localhost',
    'port': 11436
})


In [7]:
vn.connect_to_sqlite('data/tips.db')

## Training Data Import
You Obtain This Data From **Step 2**

In [8]:
vn.train(ddl='''
CREATE TABLE "tips" (
"total_bill" REAL,
  "tip" REAL,
  "sex" TEXT,
  "smoker" TEXT,
  "day" TEXT,
  "time" TEXT,
  "size" INTEGER
)
''')

Adding ddl: 
CREATE TABLE "tips" (
"total_bill" REAL,
  "tip" REAL,
  "sex" TEXT,
  "smoker" TEXT,
  "day" TEXT,
  "time" TEXT,
  "size" INTEGER
)



'834d2ae5-41fb-546f-aa01-12edf8735c5a-ddl'

## Initiate Vanna Panel
You May Access It Through http://localhost:8084

In [9]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn, allow_llm_to_see_data=True)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
